<a href="https://colab.research.google.com/github/pereira-71/-EXAMEN-01-PRIMER-PARCIAL/blob/main/ia2_1_actas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 1. Instalar dependencias
!pip install pytesseract pillow opencv-python scikit-learn seaborn pandas torch torchvision
!apt-get update
!apt-get install -y tesseract-ocr tesseract-ocr-spa

# 2. Copiar todo el código del artifact y ejecutar

# 3. Para demo completo (recomendado)
#main_demo()

# 4. Para procesar una sola acta específica
#test_single_acta()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,617 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,576 kB]
Fetched 5,577 kB in 3s (1,884 kB

In [6]:

import os
import cv2
import numpy as np
import sqlite3
import json
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
from PIL import Image, ImageDraw
import pytesseract
import re
from google.colab import drive
from google.colab import files
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import warnings
warnings.filterwarnings('ignore')

# ================================
# 1. CONFIGURACIÓN INICIAL
# ================================

class ConfiguracionSistema:
    def __init__(self):
        # Rutas principales
        self.ruta_base = '/content/drive/MyDrive/actas_electorales_2025_processed'
        self.ruta_modelos = '/content/drive/MyDrive/modelos_actas'
        self.ruta_bd = '/content/drive/MyDrive/electoral_database.db'

        # Configuración de recintos
        self.recintos_config = {
            'Campo_Deportivo_20_de_Octubre': {'prefijo': 'acta_CDP_', 'total': 7},
            'Campo_Deportivo_Barrio_Lindo': {'prefijo': 'acta_CDBL_', 'total': 4},
            'Campo_Deportivo_Noria_Alta': {'prefijo': 'acta_CDNA_', 'total': 7},
            'Campo_Deportivo_San_Juanillo_Bajo': {'prefijo': 'acta_CDSJ_', 'total': 9},
            'Campo_Deportivo_Zona_America': {'prefijo': 'acta_CDSA_', 'total': 8},
            'Campo_Deportivo_Zona_San_Francisco': {'prefijo': 'acta_CDZSF_', 'total': 7},
            'Centro_Recreacional_La_Esperanza': {'prefijo': 'acta_ZRLE_', 'total': 4},
            'Colegio_Bernardo_Monteagudo': {'prefijo': 'acta_CBM_', 'total': 6}
        }

        # Partidos políticos
        self.partidos = ['AP', 'LYP_ADN', 'APB_SUMATE', 'LIBRE', 'FP', 'MAS_IPSP', 'MORENA', 'UNIDAD', 'PDC']

        # Configuración del modelo
        self.imagen_size = (224, 224)
        self.batch_size = 8  # Reducido para Colab gratuito
        self.learning_rate = 0.001
        self.epochs_per_session = 5  # Entrenar en sesiones cortas

# ================================
# 4. MODELO DE DEEP LEARNING
# ================================

class ModeloClasificadorActas(nn.Module):
    def __init__(self, num_classes=10):  # Ajustar según número de mesas
        super(ModeloClasificadorActas, self).__init__()

        # Usar MNASNet1_0 como base (ligero para Colab gratuito)
        self.backbone = torch.hub.load('pytorch/vision:v0.10.0', 'mnasnet1_0', pretrained=True)

        # Congelar las primeras capas para transfer learning eficiente
        for param in list(self.backbone.parameters())[:-10]:
            param.requires_grad = False

        # Modificar la última capa
        self.backbone.classifier = nn.Sequential(
            nn.Linear(1280, 256),  # MNASNet1_0 tiene 1280 características
            nn.ReLU(),
            nn.Dropout(0.4),  # Aumentado para prevenir overfitting
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),  # Aumentado para prevenir overfitting
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

class DatasetActas(Dataset):
    def __init__(self, imagenes_paths, etiquetas, transform=None):
        self.imagenes_paths = imagenes_paths
        self.etiquetas = etiquetas
        self.transform = transform

    def __len__(self):
        return len(self.imagenes_paths)

    def __getitem__(self, idx):
        imagen = Image.open(self.imagenes_paths[idx]).convert('RGB')
        etiqueta = self.etiquetas[idx]

        if self.transform:
            imagen = self.transform(imagen)

        return imagen, etiqueta

# ================================
# 5. ENTRENADOR DEL MODELO
# ================================

class EntrenadorModelo:
    def __init__(self, config):
        self.config = config
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Usando dispositivo: {self.device}")

        # Crear directorio para modelos
        os.makedirs(self.config.ruta_modelos, exist_ok=True)

    def preparar_datos(self):
        """Prepara los datos de entrenamiento, validación y prueba"""
        imagenes_paths = []
        etiquetas = []

        # Recopilar todas las imágenes
        for recinto, info in self.config.recintos_config.items():
            ruta_recinto = os.path.join(self.config.ruta_base, recinto)
            if os.path.exists(ruta_recinto):
                for i in range(1, info['total'] + 1):
                    nombre_archivo = f"{info['prefijo']}{i}.jpeg"
                    ruta_imagen = os.path.join(ruta_recinto, nombre_archivo)
                    if os.path.exists(ruta_imagen):
                        imagenes_paths.append(ruta_imagen)
                        etiquetas.append(i - 1)  # Ajustar etiquetas para que comiencen en 0

        # Dividir datos: 70% entrenamiento, 10% validación, 20% prueba
        train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
            imagenes_paths, etiquetas, test_size=0.2, random_state=42, stratify=etiquetas
        )
        train_paths, val_paths, train_labels, val_labels = train_test_split(
            train_val_paths, train_val_labels, test_size=0.125, random_state=42, stratify=train_val_labels
        )  # 0.125 de 80% = 10% del total

        # Transformaciones de datos
        transform_train = transforms.Compose([
            transforms.Resize(self.config.imagen_size),
            transforms.RandomHorizontalFlip(p=0.3),  # Aumentado para más variedad
            transforms.RandomRotation(degrees=5),  # Más rotación
            transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Más variación
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        transform_val_test = transforms.Compose([
            transforms.Resize(self.config.imagen_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Crear datasets
        train_dataset = DatasetActas(train_paths, train_labels, transform_train)
        val_dataset = DatasetActas(val_paths, val_labels, transform_val_test)
        test_dataset = DatasetActas(test_paths, test_labels, transform_val_test)

        # Crear dataloaders
        train_loader = DataLoader(train_dataset, batch_size=self.config.batch_size,
                                 shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=self.config.batch_size,
                                shuffle=False, num_workers=2)
        test_loader = DataLoader(test_dataset, batch_size=self.config.batch_size,
                                shuffle=False, num_workers=2)

        return train_loader, val_loader, test_loader, len(train_dataset), len(val_dataset), len(test_dataset)

    def entrenar_sesion(self, sesion_num):
        """Entrena el modelo por una sesión limitada con early stopping"""
        print(f"\n=== INICIANDO SESIÓN DE ENTRENAMIENTO {sesion_num} ===")

        # Preparar datos
        train_loader, val_loader, test_loader, train_size, val_size, test_size = self.preparar_datos()
        print(f"Datos: {train_size} entrenamiento, {val_size} validación, {test_size} prueba")

        # Crear o cargar modelo
        num_classes = len(set().union(*[list(range(1, info['total']+1))
                                       for info in self.config.recintos_config.values()]))

        modelo = ModeloClasificadorActas(num_classes=num_classes).to(self.device)

        # Cargar modelo previo si existe
        ruta_modelo_anterior = os.path.join(self.config.ruta_modelos,
                                          f'modelo_sesion_{sesion_num-1}.pth')
        if sesion_num > 1 and os.path.exists(ruta_modelo_anterior):
            print(f"Cargando modelo de sesión anterior: {ruta_modelo_anterior}")
            modelo.load_state_dict(torch.load(ruta_modelo_anterior, map_location=self.device))

        # Configurar optimizador y función de pérdida
        criterio = nn.CrossEntropyLoss()
        optimizador = optim.Adam(modelo.parameters(), lr=self.config.learning_rate, weight_decay=0.001)
        scheduler = optim.lr_scheduler.StepLR(optimizador, step_size=3, gamma=0.7)

        # Listas para tracking
        train_losses = []
        train_accuracies = []
        val_losses = []
        val_accuracies = []

        # Early stopping
        patience = 2
        best_val_loss = float('inf')
        best_model_state = None
        counter = 0

        # Entrenamiento
        for epoca in range(self.config.epochs_per_session):
            modelo.train()
            running_loss = 0.0
            correct = 0
            total = 0

            print(f"\nÉpoca {epoca + 1}/{self.config.epochs_per_session}")

            for batch_idx, (imagenes, etiquetas) in enumerate(train_loader):
                imagenes, etiquetas = imagenes.to(self.device), etiquetas.to(self.device)

                # Forward pass
                optimizador.zero_grad()
                outputs = modelo(imagenes)
                loss = criterio(outputs, etiquetas)

                # Backward pass
                loss.backward()
                optimizador.step()

                # Estadísticas
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += etiquetas.size(0)
                correct += predicted.eq(etiquetas).sum().item()

                if batch_idx % 5 == 0:
                    print(f'Batch {batch_idx}/{len(train_loader)}, '
                          f'Loss: {loss.item():.4f}, '
                          f'Acc: {100.*correct/total:.2f}%')

            # Estadísticas de época (entrenamiento)
            epoch_loss = running_loss / len(train_loader)
            epoch_acc = 100. * correct / total
            train_losses.append(epoch_loss)
            train_accuracies.append(epoch_acc)

            # Evaluación en validación
            modelo.eval()
            val_loss = 0.0
            correct_val = 0
            total_val = 0
            with torch.no_grad():
                for imagenes, etiquetas in val_loader:
                    imagenes, etiquetas = imagenes.to(self.device), etiquetas.to(self.device)
                    outputs = modelo(imagenes)
                    loss = criterio(outputs, etiquetas)
                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    total_val += etiquetas.size(0)
                    correct_val += predicted.eq(etiquetas).sum().item()

            epoch_val_loss = val_loss / len(val_loader)
            epoch_val_acc = 100. * correct_val / total_val
            val_losses.append(epoch_val_loss)
            val_accuracies.append(epoch_val_acc)

            print(f'Época {epoca + 1} - Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%')
            print(f'              Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%')

            # Early stopping
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                best_model_state = modelo.state_dict()
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print(f"Early stopping activado en época {epoca + 1}")
                    modelo.load_state_dict(best_model_state)
                    break

            scheduler.step()

            # Guardar progreso en BD
            bd = GestorBaseDatos(self.config.ruta_bd)
            conn = sqlite3.connect(bd.ruta_bd)
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO historial_entrenamiento (sesion, epoca, accuracy, loss)
                VALUES (?, ?, ?, ?)
            ''', (sesion_num, epoca + 1, epoch_acc, epoch_loss))
            conn.commit()
            conn.close()

        # Evaluación final en prueba
        accuracy_prueba, reporte_clasificacion = self.evaluar_modelo(modelo, test_loader)

        # Guardar modelo de la sesión
        ruta_modelo_guardado = os.path.join(self.config.ruta_modelos,
                                          f'modelo_sesion_{sesion_num}.pth')
        torch.save(modelo.state_dict(), ruta_modelo_guardado)

        print(f"\nModelo guardado en: {ruta_modelo_guardado}")
        print(f"Accuracy en prueba: {accuracy_prueba:.2f}%")

        return {
            'train_losses': train_losses,
            'train_accuracies': train_accuracies,
            'val_losses': val_losses,
            'val_accuracies': val_accuracies,
            'test_accuracy': accuracy_prueba,
            'classification_report': reporte_clasificacion
        }

    def evaluar_modelo(self, modelo, test_loader):
        """Evalúa el modelo en el conjunto de prueba"""
        modelo.eval()
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for imagenes, etiquetas in test_loader:
                imagenes, etiquetas = imagenes.to(self.device), etiquetas.to(self.device)
                outputs = modelo(imagenes)
                _, predicted = outputs.max(1)

                total += etiquetas.size(0)
                correct += predicted.eq(etiquetas).sum().item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(etiquetas.cpu().numpy())

        accuracy = 100. * correct / total
        reporte = classification_report(all_labels, all_predicted)

        return accuracy, reporte

# ================================
# 6. SISTEMA PRINCIPAL (Solo la parte modificada)
# ================================

class SistemaElectoral:
    # ... (resto del código igual hasta visualizar_entrenamiento)

    def visualizar_entrenamiento(self, sesion, resultado):
        """Visualiza los resultados del entrenamiento"""
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        # Gráfico de pérdida
        axes[0].plot(resultado['train_losses'], label='Train Loss', color='red')
        axes[0].plot(resultado['val_losses'], label='Val Loss', color='orange')
        axes[0].set_xlabel('Época')
        axes[0].set_ylabel('Pérdida')
        axes[0].set_title(f'Pérdida - Sesión {sesion}')
        axes[0].legend()
        axes[0].grid(True)

        # Gráfico de accuracy
        axes[1].plot(resultado['train_accuracies'], label='Train Accuracy', color='blue')
        axes[1].plot(resultado['val_accuracies'], label='Val Accuracy', color='green')
        axes[1].axhline(y=resultado['test_accuracy'], color='purple', linestyle='--',
                       label=f'Test Accuracy: {resultado["test_accuracy"]:.2f}%')
        axes[1].set_xlabel('Época')
        axes[1].set_ylabel('Accuracy (%)')
        axes[1].set_title(f'Accuracy - Sesión {sesion}')
        axes[1].legend()
        axes[1].grid(True)

        plt.tight_layout()
        plt.show()

        print(f"\nReporte de Clasificación - Sesión {sesion}:")
        print(resultado['classification_report'])

    # ... (resto del código igual: procesar_acta_demo, visualizar_procesamiento, etc.)